In [1]:
import pandas as pd
import glob
import os
import json

def consolidate_steamspy_data():
    # Path to your steamspy CSV files
    path = 'steamspy_pages/'
    

    all_files = glob.glob(os.path.join(path, "steamspy_page_*.csv"))
    
    df_list = []
    
    # Read each file into a dataframe and append to the list
    for filename in all_files:
        try:
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f"Loaded {filename}: {len(df)} games")
        except Exception as e:
            print(f"Error loading {filename}: {e}")
    
    # Concatenate all dataframes
    combined_df = pd.concat(df_list, axis=0, ignore_index=True)
    
    combined_df = combined_df.drop_duplicates(subset=['appid'])
    
    print(f"Total games after consolidation: {len(combined_df)}")
    
    numeric_columns = ['appid', 'positive', 'negative', 'userscore', 
                        'average_forever', 'average_2weeks', 
                        'median_forever', 'median_2weeks', 
                        'price', 'initialprice', 'discount', 'ccu']
    
    for col in numeric_columns:
        if col in combined_df.columns:
            combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    
    if 'owners' in combined_df.columns:
        combined_df['owners_estimate'] = combined_df['owners'].str.split(' .. ').str[0].str.replace(',', '')
        combined_df['owners_estimate'] = pd.to_numeric(combined_df['owners_estimate'], errors='coerce')
    
    # Calculate review ratio
    combined_df['review_ratio'] = combined_df['positive'] / (combined_df['positive'] + combined_df['negative']).clip(lower=1)
    
    # Save to CSV
    combined_df.to_csv('consolidated_games.csv', index=False)
    
    return combined_df

games_df = consolidate_steamspy_data()

Loaded steamspy_pages\steamspy_page_0.csv: 1000 games
Loaded steamspy_pages\steamspy_page_1.csv: 1000 games
Loaded steamspy_pages\steamspy_page_10.csv: 1000 games
Loaded steamspy_pages\steamspy_page_11.csv: 1000 games
Loaded steamspy_pages\steamspy_page_12.csv: 1000 games
Loaded steamspy_pages\steamspy_page_13.csv: 1000 games
Loaded steamspy_pages\steamspy_page_14.csv: 1000 games
Loaded steamspy_pages\steamspy_page_15.csv: 1000 games
Loaded steamspy_pages\steamspy_page_16.csv: 1000 games
Loaded steamspy_pages\steamspy_page_17.csv: 1000 games
Loaded steamspy_pages\steamspy_page_18.csv: 1000 games
Loaded steamspy_pages\steamspy_page_19.csv: 1000 games
Loaded steamspy_pages\steamspy_page_2.csv: 1000 games
Loaded steamspy_pages\steamspy_page_20.csv: 1000 games
Loaded steamspy_pages\steamspy_page_21.csv: 1000 games
Loaded steamspy_pages\steamspy_page_22.csv: 1000 games
Loaded steamspy_pages\steamspy_page_23.csv: 1000 games
Loaded steamspy_pages\steamspy_page_24.csv: 1000 games
Loaded steams

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Load collected user data
with open('./users/user_data_730.json', 'r') as f:
    user_data = json.load(f)

# Create interactions dataframe
interactions = []
for user in user_data:
    steam_id = user['steam_id']
    for game in user['response']['games']:
        interactions.append({
            'user_id': steam_id,
            'game_id': game['appid'],
            'playtime_forever': game['playtime_forever'],
            'playtime_2weeks': game.get('playtime_2weeks', 0)
        })

interactions_df = pd.DataFrame(interactions)

# Filter interactions to include only games in our games_df
valid_game_ids = set(games_df['appid'].values)
interactions_df = interactions_df[interactions_df['game_id'].isin(valid_game_ids)]

print(f"Total interactions: {len(interactions_df)}")
print(f"Unique users: {interactions_df['user_id'].nunique()}")
print(f"Unique games: {interactions_df['game_id'].nunique()}")

Total interactions: 162134
Unique users: 690
Unique games: 14122


In [4]:
interactions_df.groupby('game_id').count().sort_values('user_id', ascending= False)

,user_id,playtime_forever,playtime_2weeks
game_id,,,
730,531,531,531
582010,495,495,495
550,463,463,463
230410,447,447,447
1085660,434,434,434
105600,432,432,432
1172470,423,423,423
431960,407,407,407
1245620,400,400,400


# Making the graph

Two components to our graph:
1. Users
2. Games

Users will be connected to other users. For simplicity sake, we can make this N = 5 number of users. This represents the top 5 most similar users.\
Games will be connected to other games. For simplicity sake, we can make this M = 5 number of games. This represents the top 5 most similar games.

In [51]:
df_tagged = pd.read_csv('steamspy_page_0_tags_final.csv')
for index in range(1,5):
    df_cur = pd.read_csv(f'steamspy_page_{index}_tags_final.csv')
    df_tagged = pd.concat([df_tagged, df_cur])
df_tagged.shape

(5000, 5)

In [52]:
df_final = pd.merge(games_df, df_tagged, on='appid')
df_final.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,owners_estimate,review_ratio,categories,genres,short_description,detailed_description
0,570,Dota 2,Valve,Valve,NaN,2007768,454710,0,"200,000,000 .. 500,000,000",0,0,0,0,0.0,0.0,0.0,572407,200000000,0.815345,"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']","Every day, millions of players worldwide enter...",<strong>The most-played game on Steam.</strong...
1,730,Counter-Strike: Global Offensive,Valve,Valve,NaN,7537874,1146419,0,"100,000,000 .. 200,000,000",0,0,0,0,0.0,0.0,0.0,1286849,100000000,0.867989,"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer..."
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",NaN,1498320,1028359,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,561947,50000000,0.593000,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...","PUBG: BATTLEGROUNDS, the high-stakes winner-ta...","<p class=""bb_paragraph""><img class=""bb_img"" sr..."
3,1623730,Palworld,Pocketpair,Pocketpair,NaN,353078,21994,0,"50,000,000 .. 100,000,000",0,0,0,0,2999.0,2999.0,0.0,43501,50000000,0.941361,"['Single-player', 'Multi-player', 'Co-op', 'On...","['Action', 'Adventure', 'Indie', 'RPG', 'Early...","Fight, farm, build and work alongside mysterio...",Q. What kind of game is this?<br><br>A. In thi...
4,1172470,Apex Legends,Respawn,Electronic Arts,NaN,662128,323698,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,129980,50000000,0.671648,"['Multi-player', 'PvP', 'Online PvP', 'Co-op',...","['Action', 'Adventure', 'Free To Play']","Apex Legends is the award-winning, free-to-pla...","<h1>Apex Legends: Takeover</h1><p><img class=""..."


In [53]:
df_final = df_final.drop(columns = ['score_rank'])
df_final = df_final[~df_final.isnull().any(axis=1)]
df_final.head()

,appid,name,developer,publisher,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,owners_estimate,review_ratio,categories,genres,short_description,detailed_description
0,570,Dota 2,Valve,Valve,2007768,454710,0,"200,000,000 .. 500,000,000",0,0,0,0,0.0,0.0,0.0,572407,200000000,0.815345,"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']","Every day, millions of players worldwide enter...",<strong>The most-played game on Steam.</strong...
1,730,Counter-Strike: Global Offensive,Valve,Valve,7537874,1146419,0,"100,000,000 .. 200,000,000",0,0,0,0,0.0,0.0,0.0,1286849,100000000,0.867989,"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer..."
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",1498320,1028359,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,561947,50000000,0.593000,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...","PUBG: BATTLEGROUNDS, the high-stakes winner-ta...","<p class=""bb_paragraph""><img class=""bb_img"" sr..."
3,1623730,Palworld,Pocketpair,Pocketpair,353078,21994,0,"50,000,000 .. 100,000,000",0,0,0,0,2999.0,2999.0,0.0,43501,50000000,0.941361,"['Single-player', 'Multi-player', 'Co-op', 'On...","['Action', 'Adventure', 'Indie', 'RPG', 'Early...","Fight, farm, build and work alongside mysterio...",Q. What kind of game is this?<br><br>A. In thi...
4,1172470,Apex Legends,Respawn,Electronic Arts,662128,323698,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,129980,50000000,0.671648,"['Multi-player', 'PvP', 'Online PvP', 'Co-op',...","['Action', 'Adventure', 'Free To Play']","Apex Legends is the award-winning, free-to-pla...","<h1>Apex Legends: Takeover</h1><p><img class=""..."


### Blow up the categories genres

Categories and genres are currently unpacked lists. We can 'one hot' encode them.

In [56]:
categories = df_final['categories'].apply(eval)

In [57]:
categories = categories.explode()
categories.to_list()
categories = list(set(categories))
# print(len(categories))
print(categories)

['Supporto parziale per i controller', 'MMO', '包括 Source SDK', '解説モード使用可能', '包含关卡编辑器', 'Tracked Controller Support', 'オンライン協力プレイ', 'Single-player', 'Steam 排行榜', 'Remote Play para móviles', '单人', 'Кросс-платформенный мультиплеер', '已启用 Valve 反作弊保护', 'Полная поддержка контроллеров', 'Stats', 'Compat. total con mando', 'Steam 成就', 'HDR available', '线上玩家对战', 'Includes Source SDK', nan, 'Семейный доступ', 'Commentary available', 'Cartas Colecionáveis Steam', 'Steam Turn Notifications', 'Таблицы лидеров Steam', 'Steam実績', 'Compat. total con control', 'Mods', 'テレビでRemote Play', 'Remote Play Together', nan, nan, '在平板上远程畅玩', 'Multijugador multiplataforma', 'Steam Workshop', 'Family Sharing', '解说可用', 'Cooperativos en línea', 'Cooperativo on-line', 'Steam Trading Cards', 'Cooperativo', 'Steamワークショップ', nan, nan, nan, '远程同乐', 'Внутриигровые покупки', 'レベル編集可能', 'キャプション使用可能', '家庭共享', 'Steamクラウド', 'Co-op', 'Nuvem Steam', 'Valveアンチチート有効', 'Cross-Platform Multiplayer', 'Remote Play на телевизоре', 'Rem

In [ ]:
# Giant JSON of curse
category_map = {
    "部分支持控制器": "Partial Controller Support",
    "Игрок против игрока (по сети)": "Online PvP",
    "JcJ en línea": "Online PvP",
    "Cooperativo en línea": "Online Co-op",
    "Stats": "Stats",
    "VR Supported": "VR Support",
    "JcJ": "PvP",
    "Steam Leaderboards": "Steam Leaderboards",
    "Steam Trading Cards": "Steam Trading Cards",
    "已启用 Valve 反作弊保护": "Valve Anti-Cheat enabled",
    "Carte collezionabili di Steam": "Steam Trading Cards",
    "HDR available": "HDR Available",
    "Внутриигровые покупки": "In-App Purchases",
    "解说可用": "Commentary Available",
    "Family Sharing": "Family Sharing",
    "Compat. total com controle": "Full Controller Support",
    "オンライン協力プレイ": "Online Co-op",
    "キャプション使用可能": "Captions Available",
    "フルコントローラサポート": "Full Controller Support",
    "Общий/разделённый экран": "Shared/Split Screen",
    "Steam 集换式卡牌": "Steam Trading Cards",
    "解説モード使用可能": "Commentary Mode Available",
    "Commentary available": "Commentary Available",
    "Valve Anti-Cheat enabled": "Valve Anti-Cheat enabled",
    "LAN Co-op": "LAN Co-op",
    "Captions available": "Captions Available",
    "Steam 成就": "Steam Achievements",
    "Steam Timeline": "Steam Timeline",
    "Shared/Split Screen PvP": "Shared/Split Screen PvP",
    "Коллекционные карточки Steam": "Steam Trading Cards",
    "Online PvP": "Online PvP",
    "玩家对战": "Player vs Player",
    "ファミリーシェアリング": "Family Sharing",
    "Steamトレーディングカード": "Steam Trading Cards",
    "Multijugador": "Multiplayer",
    "Remote Play on Phone": "Remote Play on Phone",
    "在平板上远程畅玩": "Remote Play on Tablet",
    "Conquistas Steam": "Steam Achievements",
    "Полная поддержка контроллеров": "Full Controller Support",
    "Семейный доступ": "Family Sharing",
    "データ": "Stats",
    "Remote Play на телевизоре": "Remote Play on TV",
    "Giocatore singolo": "Single-player",
    "Compat. parcial com controle": "Partial Controller Support",
    "Um jogador": "Single-player",
    "支持字幕": "Captions Available",
    "協力プレイ": "Co-op",
    "In-App Purchases": "In-App Purchases",
    "Logros de Steam": "Steam Achievements",
    "在线合作": "Online Co-op",
    "Partial Controller Support": "Partial Controller Support",
    "Un jugador": "Single-player",
    "Valveアンチチート有効": "Valve Anti-Cheat enabled",
    "PvP": "PvP",
    "Remote Play para tabletas": "Remote Play on Tablet",
    "Steam実績": "Steam Achievements",
    "Steam 排行榜": "Steam Leaderboards",
    "Shared/Split Screen Co-op": "Shared/Split Screen Co-op",
    "Compras dentro de la aplicación": "In-App Purchases",
    "合作": "Co-op",
    "Steam Cloud": "Steam Cloud",
    "在手机上远程畅玩": "Remote Play on Phone",
    "Includes Source SDK": "Includes Source SDK",
    "Tracked Controller Support": "Tracked Controller Support",
    "Steam Turn Notifications": "Steam Turn Notifications",
    "Remote Play no tablet": "Remote Play on Tablet",
    "Mods (require HL2)": "Mods (require HL2)",
    "Steam 创意工坊": "Steam Workshop",
    "完全支持控制器": "Full Controller Support",
    "同屏/分屏合作": "Shared/Split Screen Co-op",
    "Compat. total con control": "Full Controller Support",
    "Steam Workshop": "Steam Workshop",
    "统计数据": "Stats",
    "レベル編集可能": "Includes Level Editor",
    "线上玩家对战": "Online PvP",
    "Remote Play on TV": "Remote Play on TV",
    "Remote Play para móviles": "Remote Play on Phone",
    "Remote Play Together": "Remote Play Together",
    "Steam 云": "Steam Cloud",
    "VR Only": "VR Only",
    "シングルプレイヤー": "Single-player",
    "Mods": "Mods",
    "多人": "Multiplayer",
    "Multi-player": "Multiplayer",
    "Compartilhamento em família": "Family Sharing",
    "LAN協力プレイ": "LAN Co-op",
    "Cooperativo": "Co-op",
    "SteamVR Collectibles": "SteamVR Collectibles",
    "Достижения Steam": "Steam Achievements",
    "Remote Play on Tablet": "Remote Play on Tablet",
    "远程同乐": "Remote Play Together",
    "Cross-Platform Multiplayer": "Cross-Platform Multiplayer",
    "Tarjetas de Steam": "Steam Trading Cards",
    "Steam 时间轴": "Steam Timeline",
    "Includes level editor": "Includes Level Editor",
    "同屏/分屏": "Shared/Split Screen",
    "HDR disponível": "HDR Available",
    "Condivisione familiare": "Family Sharing",
    "Cooperativos": "Co-op",
    "Game demo": "Game Demo",
    "Таблицы лидеров Steam": "Steam Leaderboards",
    "Shared/Split Screen": "Shared/Split Screen",
    "Multijogador": "Multiplayer",
    "Для нескольких игроков": "Multiplayer",
    "Nuvem Steam": "Steam Cloud",
    "Steamランキング": "Steam Leaderboards",
    "VR Support": "VR Support",
    "Remote Play na TV": "Remote Play on TV",
    "Игрок против игрока (общий/разделённый экран)": "Shared/Split Screen PvP",
    "家庭共享": "Family Sharing",
    "Поддержка HDR": "HDR Available",
    "LAN PvP": "LAN PvP",
    "单人": "Single-player",
    "Steamクラウド": "Steam Cloud",
    "Source SDKを含む": "Includes Source SDK",
    "Multijugador multiplataforma": "Cross-Platform Multiplayer",
    "Steamワークショップ": "Steam Workshop",
    "Статистика": "Stats",
    "Remote Play Together": "Remote Play Together",
    "Compat. total con mando": "Full Controller Support",
    "Игрок против игрока": "PvP",
    "テレビでRemote Play": "Remote Play on TV",
    "局域网玩家对战": "LAN PvP",
    "Remote Play no celular": "Remote Play on Phone",
    "Co-op": "Co-op",
    "包括 Source SDK": "Includes Source SDK",
    "Cooperativos en línea": "Online Co-op",
    "Full controller support": "Full Controller Support",
    "Кросс-платформенный мультиплеер": "Cross-Platform Multiplayer",
    "在电视上远程畅玩": "Remote Play on TV",
    "Supporto parziale per i controller": "Partial Controller Support",
    "Cartas Colecionáveis Steam": "Steam Trading Cards",
    "オンラインPvP": "Online PvP",
    "マルチプレイヤー": "Multiplayer",
    "包含关卡编辑器": "Includes Level Editor",
    "Single-player": "Single-player",
    "MMO": "MMO",
    "Для одного игрока": "Single-player",
    "Steam Achievements": "Steam Achievements",
    "Cooperativo on-line": "Online Co-op",
}

category_map = {str(k).title(): str(v).title() for k, v in category_map.items()}

In [58]:
def category_converter(word_list):
    word_list = eval(word_list)  # Convert string to list

    return [category_map[word] if word in category_map else word for word in word_list]

df_final['categories'].apply(category_converter)

0       [Multi-player, Co-op, Steam Trading Cards, Ste...
1       [Multi-player, Cross-Platform Multiplayer, Ste...
2       [Multi-player, PvP, Online PvP, Stats, Remote ...
3       [Single-player, Multi-player, Co-op, Online Co...
4       [Multi-player, PvP, Online PvP, Co-op, Online ...
5       [Multi-player, MMO, PvP, Online PvP, Co-op, On...
6       [Single-player, Steam Achievements, Full contr...
7       [Single-player, Multi-player, PvP, Online PvP,...
8       [Single-player, Multi-player, PvP, Online PvP,...
9       [Single-Player, Multiplayer, Player Vs Player,...
10      [Multi-player, Co-op, Online Co-op, Steam Achi...
11      [Single-player, Multi-player, MMO, PvP, Online...
12      [Single-player, Multi-player, PvP, Online PvP,...
13      [Single-player, Multi-player, MMO, PvP, Online...
14      [Single-player, Multi-player, PvP, Online PvP,...
15      [Single-player, Multi-player, PvP, Online PvP,...
16      [Multi-player, Cross-Platform Multiplayer, Ste...
17      [Singl

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
category_dummies = pd.DataFrame(
    mlb.fit_transform(df_final["categories"]),
    columns=mlb.classes_,
    index=df_final.index
)

In [ ]:
df_final.head()

,appid,name,developer,publisher,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,owners_estimate,review_ratio,categories,genres,short_description,detailed_description
0,570,Dota 2,Valve,Valve,2007768,454710,0,"200,000,000 .. 500,000,000",0,0,0,0,0.0,0.0,0.0,572407,200000000,0.815345,"[[, ', M, u, l, t, i, -, p, l, a, y, e, r, ', ...","['Action', 'Strategy', 'Free To Play']","Every day, millions of players worldwide enter...",<strong>The most-played game on Steam.</strong...
1,730,Counter-Strike: Global Offensive,Valve,Valve,7537874,1146419,0,"100,000,000 .. 200,000,000",0,0,0,0,0.0,0.0,0.0,1286849,100000000,0.867989,"[[, ', M, u, l, t, i, -, p, l, a, y, e, r, ', ...","['Action', 'Free To Play']","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer..."
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",1498320,1028359,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,561947,50000000,0.593000,"[[, ', M, u, l, t, i, -, p, l, a, y, e, r, ', ...","['Action', 'Adventure', 'Massively Multiplayer...","PUBG: BATTLEGROUNDS, the high-stakes winner-ta...","<p class=""bb_paragraph""><img class=""bb_img"" sr..."
3,1623730,Palworld,Pocketpair,Pocketpair,353078,21994,0,"50,000,000 .. 100,000,000",0,0,0,0,2999.0,2999.0,0.0,43501,50000000,0.941361,"[[, ', S, i, n, g, l, e, -, p, l, a, y, e, r, ...","['Action', 'Adventure', 'Indie', 'RPG', 'Early...","Fight, farm, build and work alongside mysterio...",Q. What kind of game is this?<br><br>A. In thi...
4,1172470,Apex Legends,Respawn,Electronic Arts,662128,323698,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,129980,50000000,0.671648,"[[, ', M, u, l, t, i, -, p, l, a, y, e, r, ', ...","['Action', 'Adventure', 'Free To Play']","Apex Legends is the award-winning, free-to-pla...","<h1>Apex Legends: Takeover</h1><p><img class=""..."
